# Facial Detection via Webcam

This project aims at creating a small IA able to detect portrait characteristics : mouth, nose and the face position.

As a beginning, we will take a look inside the database.

This project is made possible by using a preprossed version of the database **YouTube Faces DB**

 Lior Wolf, Tal Hassner and Itay Maoz
*Face Recognition in Unconstrained Videos with Matched Background Similarity.*
IEEE Conf. on Computer Vision and Pattern Recognition (CVPR), 2011.